## 1. Import the necessary libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
from tensorflow import keras
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## 2. Load the dataset as a pandas dataframe

In [2]:
#create dataframe with path to fer dataset csv file
#df=pd.read_csv("/content/drive/MyDrive/Python ai course/fer2013.csv")
df=pd.read_csv("./fer2013.csv")
#printing dataframe shape and first 3 columns
print(df.shape)
df.head(3)

(35887, 3)


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training


## 3. Modify dataset to extract features and labels

In [3]:
#we allot each pixels to a sepaarte column
no_of_pixels=len(list(df['pixels'][0].split(' '))) 

no_of_pixels #contains 48*48 pixels for each image

2304

In [4]:
# create a 2D numpy array where rows corresponds to images, 
#and columns corresponds to pixels. Thus, 
#order of matrix=(no of images) x (no of pixels)
arr=np.zeros((df.shape[0],no_of_pixels))

for index, row in df.iterrows():       #iterate rowwise in df
    pix=row['pixels'].split(' ')       #1D list of pixel values for each image
    arr[index]=np.array(pix,np.int64)  #store 1D list as array in corresponding rows
arr.shape

(35887, 2304)

In [5]:
#create a modified dataframe that contains the pixel values in separate columns 
df_mod=pd.DataFrame(arr)
df_mod['emotion']=df['emotion']
df_mod['Usage']=df['Usage']
df_mod.head(3)

,0,1,2,3,4,5,6,7,8,9,...,2296,2297,2298,2299,2300,2301,2302,2303,emotion,Usage
0,70.0,80.0,82.0,72.0,58.0,58.0,60.0,63.0,54.0,58.0,...,183.0,136.0,106.0,116.0,95.0,106.0,109.0,82.0,0,Training
1,151.0,150.0,147.0,155.0,148.0,133.0,111.0,140.0,170.0,174.0,...,95.0,108.0,102.0,67.0,171.0,193.0,183.0,184.0,0,Training
2,231.0,212.0,156.0,164.0,174.0,138.0,161.0,173.0,182.0,200.0,...,152.0,122.0,114.0,101.0,97.0,88.0,110.0,152.0,2,Training


In [6]:
#list out all unique emotions present that will be identified. 
#the emotions are identified using numbers(or indices) instead of actual string name.
# emotions = ('angry','disgust','fear','happy','sad','surprise','neutral')
df_mod['emotion'].unique()

array([0, 2, 4, 6, 3, 5, 1], dtype=int64)


## 4. Split dataset into train, test and validation datasets

In [7]:
#create a training dataset where usage = "Training",
#testing dataset where usage = "PublicTest",
#validation dataset where usage="PrivateTest"
df_mod.Usage.unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [8]:
dftr=df_mod[df_mod.Usage=="Training"]       #training
dfte=df_mod[df_mod.Usage=='PublicTest']     #testing
dfval=df_mod[df_mod.Usage=='PrivateTest']   #validation

#print shapes of each dataset
print(dftr.shape,"\n",dfte.shape,"\n",dfval.shape)

dftr.head(3)

(28709, 2306) 
 (3589, 2306) 
 (3589, 2306)


,0,1,2,3,4,5,6,7,8,9,...,2296,2297,2298,2299,2300,2301,2302,2303,emotion,Usage
0,70.0,80.0,82.0,72.0,58.0,58.0,60.0,63.0,54.0,58.0,...,183.0,136.0,106.0,116.0,95.0,106.0,109.0,82.0,0,Training
1,151.0,150.0,147.0,155.0,148.0,133.0,111.0,140.0,170.0,174.0,...,95.0,108.0,102.0,67.0,171.0,193.0,183.0,184.0,0,Training
2,231.0,212.0,156.0,164.0,174.0,138.0,161.0,173.0,182.0,200.0,...,152.0,122.0,114.0,101.0,97.0,88.0,110.0,152.0,2,Training


In [9]:
#from each dataset, extract the features and labels
#features(x-part) = the pixel values (dropping the 'Usage' and 'emotion' columns)
#labels(y-part) = the 'emotion' column

#training
xtr=dftr.drop(['Usage','emotion'],axis=1)
ytr=dftr['emotion']

#testing
xte=dfte.drop(['Usage','emotion'],axis=1)
yte=dfte['emotion']

#validation
xval=dfval.drop(['Usage','emotion'],axis=1)
yval=dfval['emotion']

print(xtr.shape,ytr.shape,xte.shape, yte.shape,xval.shape, yval.shape)

(28709, 2304) (28709,) (3589, 2304) (3589,) (3589, 2304) (3589,)


In [10]:
#to match the feature array dimension with the model input size, (which is 48,48,1),
#resize the feature 2D arrays into 4D array, i.e. 3D for each image

xtr=np.array(xtr).reshape(28709,48,48,1)
xte=np.array(xte).reshape(3589,48,48,1)
xval=np.array(xval).reshape(3589,48,48,1)

#normalise the pixel values in all feature datasets
xtr=xtr/255
xte=xte/255
xval=xval/255

## 5. Building CNN model

In [11]:

#creating the cnn model as a sequential model
model = keras.Sequential()

#FIRST CONVOLUTION LAYER
model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(layers.Conv2D(64, kernel_size= (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
#model.add(layers.Dropout(0.25))

#SECOND CONVOLUTION LAYER
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
#model.add(layers.Dropout(0.25))

#THIRD CONVOLUTION LAYER
model.add(layers.Conv2D(512, (3,3), activation='relu'))
model.add(layers.Conv2D(512, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
#model.add(layers.Dropout(0.25))

#Global MAXPOOLING LAYER
model.add(layers.Flatten())

#FIRST FULLY CONNECTED LAYER
model.add(layers.Dense(512, activation='relu'))

#SECOND FULLY CONNECTED LAYER
model.add(layers.Dense(512, activation='relu'))

#OUTPUT LAYER
model.add(layers.Dense(7, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 512)         5

In [12]:
#Compliling the model
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history=model.fit(xtr,ytr, validation_data=(xval,yval), batch_size=64, epochs=100)

Epoch 1/100
449/449 [==============================] - 667s 1s/step - loss: 1.8165 - accuracy: 0.2507 - val_loss: 1.8187 - val_accuracy: 0.2449
Epoch 2/100
 28/449 [>.............................] - ETA: 9:45 - loss: 1.8237 - accuracy: 0.2511

In [21]:
cnn = model.to_json()
with open("cnn_emrec1.json", "w") as json_file:
    json_file.write(cnn)
model.save_weights("cnn_emrec_weights1.h5")


In [ ]:
'''emotions=('angry','disgust','fear','happy','sad','surprise','neutral')
for i in range(0,6):
    print(len(list(dftr['Usage'][dftr.emotion==i])))
    print(len(list(dfte.emotion==i)))
    print(len(list(dfval.emotion==i)))
    print(emotions[i])'''